# 上市公司数量分析

- 使用tushare免费的基础信息数据集。(很遗憾,就在2023年5月底这几天,不免费了)
- 公司所属地区和所属行业，有超过7%的数据缺失。暂时不用这两个属性。

In [295]:
import pandas as pd
import altair as alt
import tushare as ts
from IPython.display import HTML

In [296]:
# https://tushare.pro/document/2?doc_id=25
# 获取tushare免费的上市公司基础信息。

def get_tushare() :
    token = '4e338634be7b63c511ff3b3bfb47ac535519d21e992ba57e1fabb8d1'
    pro = ts.pro_api(token)
    # 将网页上的返回值取回来
    tb = pd.read_html('https://tushare.pro/document/2?doc_id=25')
    fields =tb[1].iloc[:,0].to_list()
    df = pd.DataFrame()

    for i in ['L','D','P']:
        # 把退市的也加上去
        df = pd.concat([df, pro.stock_basic(list_status=i,fields=fields)], ignore_index=True)

    return df

def showProfile(df) :
    col_profile = pd.concat([df.dtypes, df.isna().sum()/len(df)*100, df.sample().T], axis=1 ).reset_index()
    col_profile.columns = ['列名', '类型', '缺失比%','样例']
    
    return (col_profile)

In [297]:
df = pd.read_pickle('../../tubasic.pkl')
showProfile(df)

,列名,类型,缺失比%,样例
0,ts_code,object,0.000000,603117.SH
1,symbol,object,0.000000,603117
2,name,object,0.000000,ST万林
3,area,object,11.053316,江苏
4,industry,object,11.053316,仓储物流
5,fullname,object,0.000000,江苏万林现代物流股份有限公司
6,enname,object,0.000000,"Jiangsu Wanlin Modern Logistics Co., Ltd."
7,cnspell,object,0.000000,stwl
8,market,object,0.018577,主板
9,exchange,object,0.000000,SSE


In [300]:
def s_to_card(ser):
    sty = '''
    <style>
      .card_group {
        display: flex;
        flex-wrap: wrap;
        justify-content: center;
        text-align: center;
      }

      .card {
        display: flex;
        flex-direction: column;
        justify-content: center;
        align-items: center;
        background: #fff;
        border-radius: 10px;
        display: inline-block;
        height: 260px;
        margin: 1rem;
        width: 260px;
        box-shadow: 0 3px 6px rgba(0, 0, 0, 0.12), 0 3px 6px rgba(0, 0, 0, 0.24);
      }

      .card_title {
        border-bottom: 5px;
        border-bottom-style: dotted;
        border-bottom-color: grey;
        margin: 5px;
        padding: 5px;
        font-size: 2em;
      }
      .card_content {
        font-size: 4em;
        margin: 5px;
        align-self: center;
        height: 200px;

        display: flex;
        align-items: center;
        justify-content: center;
        justify-items: center;
      }
    </style>
    '''
    card_html=''
    for a, b in ser.items():
        card_title=a
        card_content=b
        card=f'''
        <div class="card">
            <div class="card_title">{card_title}</div>
            <div class="card_content">
            <span>{card_content}</span>
            </div>
        </div>
        '''
        card_html = card_html+card

    card_group = f'''
    {sty}
    <div class="card_group">
        {card_html}
    </div>
    '''

    return HTML(card_group)

## 上市和退市

- 至数据更新时,A股三市现存上市公司5185家
- 已经退市的198家
- 总体退市率为3.7%
- 分板块来看:主板的退市率最高,当然科创板才刚开始.
- 分交易所来看:沪深差不多,沪市稍高.

In [301]:
by_list = df.groupby('list_status')['symbol'].count()
by_list.index = ['退市','上市']
s_to_card(by_list)

In [302]:
by_list_tab = df.pivot_table(index='market', columns='list_status', values='symbol', aggfunc='count', fill_value=0, margins=True)
by_list_tab.columns = ['退市', '上市', '合计']
by_list_tab

,退市,上市,合计
market,,,
主板,177,3202,3379
创业板,17,1261,1278
北交所,3,195,198
科创板,0,527,527
All,197,5185,5382


In [303]:
by_list_tab = df.pivot_table(index='exchange', columns='list_status', values='symbol', aggfunc='count', fill_value=0, margins=True)
by_list_tab.columns = ['退市', '上市', '合计']
by_list_tab

,退市,上市,合计
exchange,,,
BSE,3,195,198
SSE,93,2213,2306
SZSE,102,2777,2879
All,198,5185,5383


In [304]:
delist_by_year = df[df.list_status == 'D'].groupby(df.delist_date.str[0:4])['symbol'].count().to_frame('delist_num').reset_index()
alt.Chart(delist_by_year).mark_line(point={'radius':8, 'fill':'white', 'stroke':'grey'}).encode(
    alt.X(delist_by_year.columns[0]),
    alt.Y(delist_by_year.columns[1]),
    tooltip=['delist_date', 'delist_num']
).properties(
    width='container',
    title='退市公司数量年度变化')


alt.Chart(...)

- 近三年(2020-2022)退市公司数连创新高,2023年不知道情况如何.

- 退市公司上市时间的分布情况:

In [305]:
delisted = df[df.list_status == 'D'][['symbol', 'name', 'list_date', 'delist_date']]
delisted['list_date'] = pd.to_datetime(delisted['list_date'], format='%Y%m%d')
delisted['delist_date'] = pd.to_datetime(delisted['delist_date'], format='%Y%m%d')
delisted['delta'] = delisted['delist_date'] - delisted['list_date']
delisted['years'] = (delisted['delta'].dt.days / 365).round(2)
delisted.sort_values('years', inplace=True)

In [306]:
s_to_card(delisted.years.describe().round(2))


- 退市公司的上市寿命中位数为12年,平均大约14年
- 最长寿的活了31.5年,最短命的只上市了不到两年.
- 以下分别列示出top10

In [307]:
delisted[['symbol', 'name', 'years']].head(10)\
    .style.set_caption('最短命的退市公司')\
    .format(precision=2)

,symbol,name,years
5377,832317,观典防务(退),1.75
5378,833874,泰祥股份(退),1.98
5379,833994,翰博高新(退),1.99
5258,600472,包头铝业(退),2.63
5355,300372,欣泰退(退),3.59
5382,603157,退拉夏(退),4.66
5234,300186,大华农(退),4.66
5299,601268,*ST二重(退),5.30
5286,601299,中国北车(退),5.39
5193,000588,PT粤金曼(退),5.40


In [308]:
delisted[['symbol', 'name', 'years']].nlargest(10, columns='years')\
    .style.set_caption('最长寿的退市公司')\
    .format(precision=2)

,symbol,name,years
5189,600652,退游久(退),31.53
5196,000502,绿景退(退),29.61
5372,600614,退鹏起(退),28.92
5289,600695,退绿庭(退),28.58
5354,600634,退富控(退),28.40
5232,600856,退中天(退),28.18
5248,000018,神城A退(退),27.58
5219,600677,*ST航通(退),27.49
5325,600870,退厦华(退),27.35
5290,600687,退刚泰(退),27.33


## 上市公司分类分析
- 目前没有退市的上市公司,分板块,分市场,分行业情况.


In [309]:
def plot_bar_percent(df) :
    df['help'] = 'help'
    cols = df.columns
    base = alt.Chart(df).encode(
        alt.X(field=cols[1], aggregate='sum', type='quantitative').axis(None).stack('zero'),    
        alt.Y('help').axis(None),
        
        alt.Order(field=cols[1], sort='descending'),
        tooltip=[cols[0], cols[1]]
    ).properties(
        width='container', height=80
    )

    bar = base.mark_bar(cornerRadius=10).encode(
        alt.Color(cols[0]).legend(orient='top'),
    )

    text = base.mark_text(
        align='center',
        baseline='line-top',
        angle=45,
        fontSize=20,
        color='white',
        dx=-15
    ).encode(
        alt.Text(field=cols[1]),  
    )

    fig = (bar+text).configure_axis(
        grid=False
    ).configure_view(
        stroke=None
    )

    return fig

In [310]:
def plot_multibar_percent(df) :
    cols = df.columns
    base = alt.Chart(df).encode(
        alt.X(field=cols[-1], aggregate='sum', type='quantitative').axis(None).stack('zero'),    
        alt.Y(cols[0]),
        
        alt.Order(field=cols[1], sort='descending'),
        tooltip=[cols[0], cols[1],cols[2]]
    ).properties(
        width='container', height=80*len(df[cols[0]].unique())
    )

    bar = base.mark_bar(cornerRadius=10).encode(
        alt.Color(cols[1]).legend(orient='top'),
    )

    text = base.mark_text(
        align='right',
        baseline='top',
        # angle=45,
        fontSize=20,
        color='white',
        dx=-15
    ).encode(
        alt.Text(field=cols[2]),  
    )

    fig = (bar+text).configure_axis(
        grid=False
    ).configure_view(
        stroke=None
    )

    return fig

## 交易所和板块
- 三个交易所,四个板块
- 从交易所上市数量来看,深交所规模最大,北交所成立较晚规模最小
- 主板规模最大

In [311]:
sel = df[df.list_status=='L']
by_exchange = df[df.list_status == 'L'].groupby('exchange')['symbol'].count().to_frame('Num').reset_index()
plot_bar_percent(by_exchange)

alt.LayerChart(...)

In [312]:
# make a stackbar to show the data
by_market = df[df.list_status == 'L'].groupby('market')['symbol'].count().to_frame('Num').reset_index()
plot_bar_percent(by_market)


alt.LayerChart(...)

In [313]:
exch_mark = sel.groupby(['exchange', 'market'])['symbol'].count().to_frame('Num').reset_index()
plot_multibar_percent(exch_mark)

alt.LayerChart(...)

- 北交所没有区分板块
- 上交所交易主板和科创板
- 深交所交易主板和创业板

## 沪港通和深港通
- 差不多一半的上市股票已经和港交所互通
- 深交所的深港通数量更多一点,但上交所的沪港通的比例较高
- 北交所没有开通

In [314]:
by_hs = df[df.list_status == 'L'].groupby('is_hs')['symbol'].count().to_frame('Num').reset_index()
plot_bar_percent(by_hs)

alt.LayerChart(...)

In [315]:
exchange_hs = sel.groupby(['exchange', 'is_hs'])['symbol'].count().to_frame('Num').reset_index()
exchange_hs['is_hs'].replace({'N':'未通', 'S':'内港通', 'H':'内港通'}, inplace=True)
plot_multibar_percent(exchange_hs)

alt.LayerChart(...)

## 年度变化


In [316]:
df['list_year'] = df['list_date'].str[0:4]
df.columns

Index(['ts_code', 'symbol', 'name', 'area', 'industry', 'fullname', 'enname',
       'cnspell', 'market', 'exchange', 'curr_type', 'list_status',
       'list_date', 'delist_date', 'is_hs', 'list_year'],
      dtype='object')

In [317]:
data = df.groupby('list_year')['symbol'].count().to_frame(name='num').reset_index()
alt.Chart(data).mark_line(point=True).encode(
    alt.X('list_year', type='ordinal'),
    alt.Y('num', type='quantitative'),
    tooltip=['list_year', 'num']
).properties(
    width='container',
    title='新增上市公司数量（2023年数据至5月）'
)

alt.Chart(...)

In [318]:
data = df.groupby(['list_year', 'market'])['symbol'].count().to_frame(name='num').reset_index()
alt.Chart(data).mark_line(point=True).encode(
    alt.X('list_year', type='ordinal'),
    alt.Y('num', type='quantitative'),
    alt.Color('market'),
    tooltip=['list_year', 'num']
).properties(
    width='container',
    title='按板块新增上市公司数量（2023年数据至5月）'
)

alt.Chart(...)

In [319]:
data = df.groupby(['list_year', 'exchange'])['symbol'].count().to_frame(name='num').reset_index()
alt.Chart(data).mark_line(point=True).encode(
    alt.X('list_year', type='ordinal'),
    alt.Y('num', type='quantitative'),
    alt.Color('exchange'),
    tooltip=['list_year', 'num']
).properties(
    width='container',
    title='按交易所新增上市公司数量（2023年数据至5月）'
)

alt.Chart(...)

- 2021年新上市公司创新高，达到524家。
- 2022年仍处于历史第3,达到428家。上一个新发高峰是2017年438家。
- 从板块看，2017年主板新发高峰，此后就是创业板和科创板增长较快了。
- 从交易所看，深交所在2010年有个大跃进，近几年来沪深数量基本相当。